In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("data/train_combine.csv")
train.head()

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,5,NaN,NaN,17.24,3648.0,-0.0711,NaN,1.9,696.0,3.6


In [3]:
y = train['x_e_out [-]']
X = train.drop(columns=['id','x_e_out [-]'])

In [4]:
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline

In [5]:
# set up preprocessing for numeric columns
imp_knn = KNNImputer(n_neighbors=1000)
imp_median = SimpleImputer(strategy='median', add_indicator=True)
scaler = StandardScaler()

In [6]:
# set up preprocessing for categorical columns
imp_constant = SimpleImputer(strategy='constant')
ohe = OneHotEncoder(handle_unknown='ignore')

In [7]:
from sklearn.compose import make_column_selector, make_column_transformer

In [8]:
# select columns by data type
num_cols = make_column_selector(dtype_include='number')
cat_cols = make_column_selector(dtype_exclude='number')

In [9]:
# do all preprocessing
preprocessor = make_column_transformer(
    (make_pipeline(imp_knn, scaler), num_cols),
    (make_pipeline(imp_constant, ohe), cat_cols))

preprocessor1 = make_column_transformer(
    (make_pipeline(imp_median, scaler), num_cols),
    (make_pipeline(imp_constant, ohe), cat_cols))

In [49]:
#记录k=3时的得分
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor,HistGradientBoostingRegressor
rfr = RandomForestRegressor() #0.372568
#abr = AdaBoostRegressor() #0.372568
hgbr = HistGradientBoostingRegressor(max_iter=1000,random_state=318) #0.0054793
gbr = GradientBoostingRegressor(n_estimators=750) #500:0.0055510

from catboost import CatBoostRegressor
cb = CatBoostRegressor(n_estimators=1000,random_state = 318,verbose=0, objective='RMSE') #0.005509

import lightgbm as lgb
lgbm = lgb.LGBMRegressor(n_estimators=125,random_state = 318) #0.005465

import xgboost as xgb
xgb = xgb.XGBRegressor(n_estimators=35,random_state = 318) #0.005579

from sklearn.ensemble import VotingRegressor
vr = VotingRegressor([('cb',cb),('gbr',gbr),('lgbm',lgbm),('xgb',xgb),('hgbr',hgbr)],weights=(1,1,1,1,1))

In [50]:
pipe = make_pipeline(preprocessor1, vr)

In [51]:
pipe.steps

[('columntransformer',
  ColumnTransformer(transformers=[('pipeline-1',
                                   Pipeline(steps=[('simpleimputer',
                                                    SimpleImputer(add_indicator=True,
                                                                  strategy='median')),
                                                   ('standardscaler',
                                                    StandardScaler())]),
                                   <sklearn.compose._column_transformer.make_column_selector object at 0x0000025D5A034A00>),
                                  ('pipeline-2',
                                   Pipeline(steps=[('simpleimputer',
                                                    SimpleImputer(strategy='constant')),
                                                   ('onehotencoder',
                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                   <sklearn.com

In [52]:
#baseline of voting_regressor
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X, y, scoring='neg_mean_squared_error').mean()

-0.005433040121326995

In [53]:
from sklearn.model_selection import GridSearchCV

params = {}
params['votingregressor__weights'] = [(1,1,2,1,2),(1,1,1,1,2),(1,1,2,1,1)]

grid = GridSearchCV(pipe, params, scoring='neg_mean_squared_error')
%time grid.fit(X,y)

Wall time: 5min 25s


GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(add_indicator=True,
                                                                                                        strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000025D5A034A00>),
                                                                        ('pipeline-2',
                           

In [54]:
grid.best_params_

{'votingregressor__weights': (1, 1, 2, 1, 2)}

In [55]:
grid.best_score_

-0.005428086652798978

In [56]:
#处理test值
test = pd.read_csv('data/test.csv')
X_test = test.drop(columns=['id','x_e_out [-]'])

In [57]:
result = grid.predict(X_test)

In [58]:
result

array([-0.00235922, -0.09811642, -0.04951853, ...,  0.0356725 ,
       -0.06226163, -0.01770191])

In [59]:
pd.DataFrame({'id':test.id, 'x_e_out [-]':result}).set_index('id').to_csv('sub_combine_data_vr_xgb_lgbm_gbr_cb_hgbr.csv')

- 0.076014